<a href="https://colab.research.google.com/github/toinnn/Torax_Xray-Health-Care/blob/main/Prepare_data_2_Yolov8__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from torchvision.io import decode_png, read_image , ImageReadMode
from PIL   import   Image
import polars as pl
import torch


path_project  = "/content/drive/Othercomputers/Meu laptop/Dataset_work_space/Torax_Xray"
image_dir_path = path_project + "/images/"
data_intro_path = path_project + "/Data_Entry_2017.csv"
label2class_path = path_project + "/target_dict_label2class.json"
BB_Original_path  = path_project + "/BBox_List_2017.csv"
BB_YoloFormat_path = path_project + "/Yolo_BBox_List_2017.csv"


def load_image_nvjpngl_gpu(image_path):
    """
    Loads an image from the specified file path using NVJPEG decoder on GPU and returns a PyTorch tensor.
    Args:
        image_path (str): The path to the image file.
    Returns:
        torch.Tensor: The PyTorch tensor representing the image.
    """
    data = read_image(image_path , mode = ImageReadMode.GRAY )
    return data.float()

    tensor = decode_png(data).float()
    return tensor

def BB_NIH_2_YOLO_Format(bb):
    x , y , w , h = bb
    x_new = x + w/2
    y_new = y + h/2

    return x_new , y_new , w , h

In [10]:
load_image_nvjpngl_gpu('/content/drive/Othercomputers/Meu laptop/Dataset_work_space/Torax_Xray/images/00005762_004.png')

tensor([[[  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         ...,
         [ 97., 163., 152.,  ...,  44.,  46.,  26.],
         [106., 179., 165.,  ...,  47.,  49.,  27.],
         [ 53.,  89.,  81.,  ...,  23.,  23.,  13.]]])

In [15]:
# Data_Entry  = pl.read_csv(Data_Entry_path)
# Data_Entry = {row['Image Index'] : row['Finding Labels'].split("|") for row in Data_Entry[['Image Index' ,'Finding Labels' ]].iter_rows(named=True)}

BB_Yolo  = []
BB_Entry = pl.read_csv(BB_Original_path)

for row in BB_Entry[['Image Index' ,'Finding Label', "Bbox [x" , "y" , "w" , "h]" ]].iter_rows(named=True) :
    BB_Yolo += [ {'Image Index' : row['Image Index'] , 'Finding Label' : row['Finding Label'] , "Bounding Box" : "{},{},{},{}".format(*( BB_NIH_2_YOLO_Format(  (row["Bbox [x"]  , row["y"] , row["w"] , row["h]"])  ) )) } ]

BB_Yolo = pl.DataFrame(BB_Yolo)
BB_Yolo.write_csv( BB_YoloFormat_path )

In [16]:
print(BB_Yolo)

shape: (984, 3)
┌──────────────────┬───────────────┬───────────────────────────────────┐
│ Image Index      ┆ Finding Label ┆ Bounding Box                      │
│ ---              ┆ ---           ┆ ---                               │
│ str              ┆ str           ┆ str                               │
╞══════════════════╪═══════════════╪═══════════════════════════════════╡
│ 00013118_008.png ┆ Atelectasis   ┆ 268.47457627118655,586.612437102… │
│ 00014716_007.png ┆ Atelectasis   ┆ 778.8474576271185,288.2892611228… │
│ 00029817_009.png ┆ Atelectasis   ┆ 299.38983050847446,425.527691340… │
│ 00014687_001.png ┆ Atelectasis   ┆ 796.745762711864,522.61243710275… │
│ …                ┆ …             ┆ …                                 │
│ 00025769_001.png ┆ Atelectasis   ┆ 753.607118055556,604.34963541666… │
│ 00016837_002.png ┆ Atelectasis   ┆ 276.87822916666653,706.180746527… │
│ 00020124_003.png ┆ Atelectasis   ┆ 297.358229166667,632.22519097222… │
│ 00026920_000.png ┆ Atelectasis   